In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)

In [2]:
mol_str = """
0 1
O
H 1 1.0
H 1 1.0 2 104.0
symmetry c1
"""

options_dict = {
    "basis": "6-31g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0]),
    'ci_level' : 'cis',
    'ignore_coupling' : False,
    'number_of_photons' : 1,
    'natural_orbitals' : False,
    'nact_orbs' : 0,
    'nact_els' : 0
}

mol = psi4.geometry(mol_str)

psi4.set_options(options_dict)
psi4.core.set_output_file("output.dat", False)


test_pf = PFHamiltonianGenerator(
    mol_str,
    options_dict,
    cavity_dict
)

e_fci, wavefunctions = np.linalg.eigh(test_pf.H_PF)

test_pf.calc1RDMfromCIS(wavefunctions[:, 14])

# test traces of different blocks
expected_trace_Dij = 9.0
expected_trace_Dab = 1.0
expected_trace_D1 = 10.0
expected_trace_D1_spatial = 10.0

trace_Dij = np.trace(test_pf.Dij)
trace_Dab = np.trace(test_pf.Dab)
trace_D1 = np.trace(test_pf.D1)
trace_D1_spatial = np.trace(test_pf.D1_spatial)

assert np.isclose(expected_trace_D1, trace_D1)
assert np.isclose(expected_trace_D1_spatial, trace_D1_spatial)
assert np.isclose(expected_trace_Dab, trace_Dab)
assert np.isclose(expected_trace_Dij, trace_Dij)

# test trace of K1 D1 against <C|1H|C>
e1_test_rdm = np.einsum("pq,pq->", test_pf.Hspin, test_pf.D1)

# 1H @ C
temp = np.einsum("pq,q->p", test_pf.H_1E, wavefunctions[:, 14])
e1_test_wfn = np.dot(wavefunctions[:, 14].T, temp)

expected_1e_energy = e1_test_wfn - test_pf.Enuc

assert np.isclose(e1_test_rdm, expected_1e_energy)




Start SCF iterations:

Canonical RHF One-electron energy = -122.2743570184695727
CQED-RHF One-electron energy      = -122.2743570184695727
Nuclear repulsion energy          = 8.8026031303907679
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -75.9800163449004344   dE = -7.59800E+01   dRMS = 5.74391E-13
SCF Iteration   2: Energy = -75.9800163449003350   dE =  9.94760E-14   dRMS = 1.52547E-13
Total time for SCF iterations: 0.001 seconds 

QED-RHF   energy: -75.98001634 hartree
Psi4  SCF energy: -75.98001634 hartree
 Completed QED-RHF in 0.3924570083618164 seconds
 Completed 1HSO Build in 0.00031495094299316406 seconds
 Completed ERI Build in 0.009588003158569336 seconds 
 Completed 2D build in 0.0027751922607421875 seconds
 Completed 1G build in 5.888938903808594e-05 seconds
 Completed determinant list in 0.0017969608306884766 seconds 
 Completed constant offset matrix in 6.103515625e-05 seconds
 Completed Hamiltonian build in 0.08815407752990723 secon

In [6]:
eig, vec = np.linalg.eigh(test_pf.D1_spatial)
idx = eig.argsort()[::-1]   
eigenValues = eig[idx]
eigenVectors = vec[:,idx]

In [8]:
print(eigenValues)

[1.99999997e+00 1.99954097e+00 1.99685785e+00 1.97484888e+00
 1.02875233e+00 9.71247674e-01 2.51511229e-02 3.14215124e-03
 4.59025293e-04 2.65065755e-08 5.03144689e-16 3.61181360e-17
 2.55813082e-18]


In [9]:
print(eigenVectors)

[[ 9.99938641e-01  1.10738700e-02  1.91643062e-15  2.47869978e-16
   2.88237960e-04  1.14567506e-14  4.77890046e-29 -1.24966900e-28
   2.75255115e-16 -1.88785414e-16 -2.53999332e-25 -5.83777075e-27
   2.43799889e-25]
 [ 1.10773270e-02 -9.99763039e-01  2.70720919e-14  4.23422196e-14
  -1.87392429e-02 -7.44838850e-13  2.23135299e-27 -2.06018903e-27
  -2.48503810e-14 -2.09136603e-18 -5.38185409e-27  2.19313884e-26
   4.54846840e-27]
 [ 7.21188121e-16 -4.23447199e-14  1.66935262e-12 -1.00000000e+00
  -1.58074817e-14 -2.21115212e-16 -1.88554624e-13 -3.95487129e-18
   5.09265190e-17 -1.39993376e-16 -2.74159862e-17  2.88658820e-17
   5.62257882e-18]
 [-8.06537189e-05 -1.87412859e-02 -1.99591424e-13 -1.50198100e-14
   9.99824363e-01  3.97405613e-11 -2.89818208e-27  1.30021952e-26
  -4.65838482e-16  1.52271827e-20 -2.85938150e-29  9.75246914e-28
   6.60261161e-29]
 [ 2.23229723e-15 -2.33038546e-14 -1.00000000e+00 -1.66990149e-12
  -2.00063126e-13 -1.47821618e-16 -8.52904158e-18  6.52072039e-14
